In [1]:
from omegaconf import OmegaConf
from datasets import load_data
import numpy as np
config = OmegaConf.load("./configs/default_config.yaml")

for dataset in [ 'StanfordCars', 'OxfordIIITPet', 
                'Caltech101', 'DTD' 
                'Imagenette', 'Flowers', 
                'EuroSAT', 'Aircraft',
                'CIFAR10', 'CIFAR100',]:
    
    config.dataset = dataset
    train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(False, config) 
    image, label = val_dataset[0]  # Extract the image and label
    pixels = np.array(image)
    print(dataset, len(train_dataset), len(val_dataset), len(test_dataset), min(pixels[0][0]),  max(pixels[0][0]), N)

datadir /home/mheuillet/Desktop/robust_training/data


RuntimeError: Dataset not found. You can use download=True to download it

In [1]:
from PIL import Image
import torchvision.datasets as datasets

# Initialize the Caltech101 dataset without transformations
caltech101_sample = datasets.Caltech101(root='./data', download=False)
print(len(caltech101_sample))
for i in range( len(caltech101_sample) ) :
    image, label = caltech101_sample[i]

    # Check the image mode
    # print(f"Image Mode: {image.mode}")  # Expected: 'RGB'
    if image.mode != 'RGB':
        print('issue')

8677
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue
issue

NameError: name 'config' is not defined

In [5]:
x[0]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=510x337>, 0)